In [105]:
%matplotlib notebook
# We start off with the baseline import statements we need to do the basic data manipulation and visualization.
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import calendar
from sklearn.ensemble import RandomForestClassifier

sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

#We create and set aside a copy of the data for initial exploration
housing_train = pd.read_csv('../data/train.csv')
housing = housing_train.copy()

#MISSING DATA
total = housing.isnull().sum().sort_values(ascending=False)
percent = (housing.isnull().sum()/housing.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

#CORRELATION CHECK
corr_matrix = housing.corr()
top_corr = corr_matrix['SalePrice'].sort_values(ascending = False)

In [106]:
#DROPPING SOME COLUMNS
drop = ['PoolQC', 'PoolArea','MiscFeature', 'MiscVal', 'Alley', 'Fence', 'FireplaceQu', 'Fireplaces', 'LotFrontage']
drop2 = ['Id','GarageArea','1stFlrSF','GarageYrBlt']
housing.drop(columns = drop + drop2, inplace = True)
housing['Age'] = housing['YrSold'] - housing['YearBuilt']
housing['AgeRemodel'] = housing['YrSold'] - housing['YearRemodAdd']
housing = housing[housing.AgeRemodel >= 0]
housing.drop(columns = ['YearBuilt','YearRemodAdd'], inplace = True)

In [107]:
#FURTHER DATA CLEANING
housing_cat = housing.select_dtypes(exclude=[np.number])
housing_numeric = housing.select_dtypes(include=[np.number])

#Numeric
numeric_unbounded = ['LotArea', 'MasVnrArea','BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF',
                     'TotalBsmtSF','2ndFlrSF','LowQualFinSF','GrLivArea','WoodDeckSF',
                     'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch', 'SalePrice',
                     'Age','AgeRemodel']

numeric_one_hot = ['MSSubClass','MoSold']

numeric_ordinal = [x for x in housing_numeric.columns 
                   if (x not in numeric_unbounded and x not in numeric_one_hot)]

housing_numeric_unbounded = housing_numeric[numeric_unbounded]
housing_numeric_one_hot = housing_numeric[numeric_one_hot]
housing_numeric_ordinal = housing_numeric[numeric_ordinal]

housing_numeric_one_hot['MSSubClass'] = housing_numeric_one_hot['MSSubClass'].astype('str')
housing_numeric_one_hot['MoSold'] = housing_numeric_one_hot['MoSold'].replace({i:calendar.month_name[i][:3] for i in range(1,13)})
housing_numeric_one_hot = pd.get_dummies(housing_numeric_one_hot)

#Categorical
cat_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
               'BsmtFinType1', 'HeatingQC', 'KitchenQual','Functional','GarageFinish',
               'GarageQual', 'GarageCond']

housing_cat_ordinal = housing_cat[cat_ordinal]
housing_cat_ordinal.fillna('None', inplace = True)
housing_cat_one_hot = housing_cat.drop(columns = cat_ordinal)
housing_cat_one_hot = pd.get_dummies(housing_cat_one_hot)

def mapper(cat):
    if cat in ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
               'HeatingQC', 'KitchenQual']:
        mapper = {'None':0, 'Po':1, 'Fa':2,'TA':3,'Gd':4,'Ex':5}
    elif cat == 'BsmtExposure':
            mapper = {'None':0,'No':1, 'Mn':2, 'Av':3,'Gd':4}
    elif cat == 'BsmtFinType1':
        mapper = {'None':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
    elif cat == 'Functional':
        mapper = {'Sal':0,'Sev':1,'Maj2':2,'Maj1':3,'Mod':4,'Min2':5, 'Min1':6,'Typ':7}
    else:
        mapper = {'None':0,'Unf':1,'RFn':2,'Fin':3}
        
    return mapper

for cat in cat_ordinal:
    housing_cat_ordinal[cat].replace(mapper(cat), inplace = True)

#Combining numeric and categorical
housing_ordinal = pd.concat([housing_numeric_ordinal,housing_cat_ordinal], axis = 'columns')
housing_one_hot = pd.concat([housing_numeric_one_hot, housing_cat_one_hot], axis = 'columns')
housing_clean = pd.concat([housing_one_hot, housing_ordinal, housing_numeric_unbounded], 
                          axis = 'columns')

#MORE CORRELATION
ordinal_prices = pd.concat([housing_ordinal, housing['SalePrice']], axis = 'columns')
ordinal_corr_matrix = ordinal_prices.corr()
top_corr_ordinal = ordinal_corr_matrix['SalePrice'].sort_values(ascending = False)

one_hot_prices = pd.concat([housing_one_hot, housing['SalePrice']], axis = 'columns')
one_hot_corr_matrix = one_hot_prices.corr()
top_corr_one_hot = one_hot_corr_matrix['SalePrice'].filter(like = 'Neighborhood').sort_values(ascending = False)

In [115]:
#REMOVAL OF THE REMAINING NaN
housing_clean.isnull().sum().sort_values(ascending=False)
df = housing_clean.copy()
problem_col = df.isin([np.nan, np.inf, -np.inf]).sum(axis=0)[df.isin([np.nan, np.inf, -np.inf]).sum(axis=0) != 0] 
index_to_drop = df[problem_col.index[0]][df[problem_col.index[0]].isin([np.nan, np.inf, -np.inf])].index
df.drop(index = index_to_drop, inplace = True)
# df.isnull().sum().sort_values(ascending=False)
df.to_csv('clean_data.csv')

In [109]:
#RANDOM FOREST FOR FEATURE IMPORTANCE
X_train = df.drop(columns = ['SalePrice'])
y_train = df['SalePrice']

forest = RandomForestClassifier(n_estimators=500, max_depth=4)

forest.fit(X_train, y_train)

forest.feature_importances_
score_df = pd.DataFrame({'feature':X_train.columns,
                            'importance_score': forest.feature_importances_})

In [110]:
#We will look at feature importances and their correlation with the 'SalePrice'
score_df.sort_values('feature', inplace=True)
top_corr = df.corr()['SalePrice'].abs().drop(index = ['SalePrice']) #I suppose we want to look at the absolute value
                                                                    #of the correlation. Is that right?
top_corr.sort_index(inplace=True) 
#now rows of score_df and top_corr match and we can add the values of correlation
score_df['correlation'] = top_corr.values

In [111]:
score_df.sort_values(ascending=False, by = ['importance_score'], inplace = True)
score_df['importance_score_rank'] = [k for k in range(1,1+len(score_df.index))]
score_df.sort_values(ascending=False, by = ['correlation'], inplace = True)
score_df['correlation_rank'] = [k for k in range(1,1+len(score_df.index))]
score_df['overall_rank'] = (score_df['importance_score_rank'] + score_df['correlation_rank'])/2 
# score_df.sort_values(ascending=True, by = ['overall_rank'], inplace = True)
score_df.sort_values(ascending=True, by = ['correlation_rank'], inplace = True)
score_df.reset_index(drop = True, inplace = True)
score_df.head(20)

,feature,importance_score,correlation,importance_score_rank,correlation_rank,overall_rank
0,OverallQual,0.02,0.79,5,1,3.00
1,GrLivArea,0.04,0.72,1,2,1.50
2,ExterQual,0.01,0.68,21,3,12.00
3,KitchenQual,0.01,0.66,18,4,11.00
4,GarageCars,0.02,0.64,15,5,10.00
5,TotalBsmtSF,0.03,0.62,4,6,5.00
6,BsmtQual,0.01,0.58,16,7,11.50
7,FullBath,0.02,0.56,9,8,8.50
8,GarageFinish,0.01,0.55,19,9,14.00
9,TotRmsAbvGrd,0.02,0.54,10,10,10.00


Correlation measures only linear dependence between variables and it does not detect non-linear dependence (in particular cor(X,Y) can be 0 for random variables X and Y=X^2, which are of course completely dependent). So, if a given feature has high feature_importance score, but low correlation it means that 'SalePrice' depend on it in a non-linear manner.

In [63]:
# plt.figure(figsize=(7, 7))
# plt.scatter(score_df['importance_score_rank'][:20], score_df['correlation_rank'][:20], c ="blue",
#             linewidths = 1)
# plt.title('importance_rank vs correlation_rank')
# plt.xticks(np.arange(0, 40, step=1))
# plt.yticks(np.arange(0, 40, step=1))
# plt.xlabel("importance_score_rank")
# plt.ylabel("correlation_rank")
# plt.show()

FITTING MODELS

In [112]:
from sklearn import linear_model
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [113]:
def eval(y_hat, y):
#     print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y)))
#     print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat - y) ** 2))
#     print( f'Normalized sum of squares error: {round(100*np.mean((y_hat - y) ** 2) / (np.mean(y ** 2)), 2)}%' )
#     print("R2-score: %.2f" % r2_score(y_hat, y) )
    return [round(np.mean(np.absolute(y_hat - y)), 2), round(np.mean((y_hat - y) ** 2),2), 
            round(np.mean((y_hat - y) ** 2) / (np.mean(y ** 2)),2), 
            round(r2_score(y_hat, y),2)]

In [114]:
#Train-test split
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

metrics = {}

#MODEL 1: Simple linear regression
print('=====================')
print('Simple linear regression')
print('=====================')

#a) Data preparation
X_train = train['GrLivArea'].values.reshape(-1,1)
y_train = train['SalePrice']
X_test = test['GrLivArea'].values.reshape(-1,1)
y_test = test['SalePrice']

#b) Fitting the model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print ('Coefficients in simple linear regression: ', [round(num,2) for num in regr.coef_])
print ('Intercept in simple linear regression: ', round(regr.intercept_,2))

#c) Drawing a graph
# plt.scatter(train.GrLivArea, train.SalePrice,  color='blue')
# plt.plot(train.GrLivArea, regr.coef_*train.GrLivArea + regr.intercept_, '-r')
# plt.title('GrLivArea vs SalePrice')
# plt.xlabel("GrLivArea")
# plt.ylabel("SalePrice")

#d) Prediction and evaluation
y_test_hat = regr.predict(X_test)
metrics['SLR'] = eval(y_test_hat, y_test )
print('Variance score: %.2f' % regr.score(X_test, y_test))

#MODEL 2: Multi-linear regression with all features
print('=====================')
print('Multi-linear regression with all features')
print('=====================')
#a) Data preparation
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#b) Fitting the model
ml_regr = linear_model.LinearRegression()
ml_regr.fit(X_train, y_train)
print (f'20 largest coefficients in multilinear regression: {[round(num, 2) for num in sorted(ml_regr.coef_, reverse=True)[:20]]}')
print ('Intercept in multilinear regression: %.2f' % round(ml_regr.intercept_,2))

#c) Drawing a graph - not clear how to do it in this case.

#d) Prediction and evaluation
y_test_hat = ml_regr.predict(X_test)
metrics['MLR'] = eval(y_test_hat, y_test)
print('Variance score: %.2f' % ml_regr.score(X_test, y_test))

#MODEL 3: Multi-linear regression using 3 features most correlated with the SalePrice
print('=====================')
print('Multi-linear regression using 3 features most correlated with the SalePrice')
print('=====================')
#a) Data preparation
X_train = train[['GrLivArea','OverallQual', 'ExterQual']]
y_train = train['SalePrice']
X_test = test[['GrLivArea','OverallQual', 'ExterQual']]
y_test = test['SalePrice']

#b) Fitting the model
ml3_regr = linear_model.LinearRegression()
ml3_regr.fit(X_train, y_train)
print (f'Coefficients in multilinear regression:{[round(num, 2) for num in ml3_regr.coef_]}')
print ('Intercept in multilinear regression: ', round(ml3_regr.intercept_,2))

#c) Drawing a graph - not clear how to do it in this case.

#d) Prediction and evaluation
y_test_hat = ml3_regr.predict(X_test)
metrics['MLR3'] = eval(y_test_hat, y_test)

Simple linear regression
Coefficients in simple linear regression:  [113.59]
Intercept in simple linear regression:  10421.43
Variance score: 0.41
Multi-linear regression with all features
20 largest coefficients in multilinear regression: [82498.43, 68506.77, 49419.57, 46486.33, 43926.93, 38319.1, 35919.26, 29830.22, 29563.87, 25234.09, 24780.46, 23846.01, 23756.84, 23619.17, 23121.44, 22544.47, 21741.07, 20590.38, 19544.18, 19294.77]
Intercept in multilinear regression: -858608.98
Variance score: 0.63
Multi-linear regression using 3 features most correlated with the SalePrice
Coefficients in multilinear regression:[63.67, 22527.06, 31977.65]
Intercept in multilinear regression:  -160386.0


In [85]:
#MODEL 4: k nearest neighbors regression

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1, 10):
    y_test_hat = KNeighborsRegressor(x).fit(X_train,y_train).predict(X_test)
#     print('=====================')
#     print(f'KNN regression with k = {x}')
#     print('=====================')
    metrics[f'KNN_{x}']= eval(y_test_hat, y_test)

In [86]:
#MODEL 5: SVM regression

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1,5):
    y_test_hat = LinearSVR(C=1, epsilon=25000*x, max_iter=100000).fit(X_train,y_train).predict(X_test)
#     print('=====================')
#     print(f'Support vector regression with epsilon = {25000*x}')
#     print('=====================')
    metrics[f'SVM_epsilon={25000*x}'] = eval(y_test_hat, y_test)

In [97]:
#MODEL 6, 7: Decision tree and Random forest

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1,10):
    y_test_hat = DecisionTreeRegressor(max_depth = x).fit(X_train,y_train).predict(X_test)
    metrics[f'Decision_tree_max_depth={x}'] = eval(y_test_hat, y_test)
    
    y_test_hat = RandomForestRegressor(max_depth = x).fit(X_train,y_train).predict(X_test)
    metrics[f'Random_forest_max_depth={x}'] = eval(y_test_hat, y_test)

Below we will study several regularized regression model. Their common feature is that they all penalize large regression coefficients. In case of Ridge regression it is achieved by adding $\alpha\|\beta\|^2_{\ell^2}$ to the MSE. In LASSO regression one adds $\alpha\|\beta\|_{\ell^1}$ instead. Finally, in the Elastic Net approach we add a convex combination:
$\alpha\Big[r\|\beta\|^2_{\ell^2}+(1-r)\|\beta\|_{\ell^1}\Big]$.

In [92]:
#MODEL 8,9 and 10: Regularized regression models

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

scaler = StandardScaler()
scaler.fit(X_train)
X_train_rescaled = scaler.transform(X_train)
X_test_rescaled = scaler.transform(X_test)

In [93]:
#d) Prediction and evaluation

for x in range(1,10):

#With rescaling

    y_test_hat = Ridge(alpha= x/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
    metrics[f'Ridge rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)
    
    y_test_hat = Lasso(alpha= x/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
    metrics[f'Lasso rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

#Without rescaling:
    
    y_test_hat = Ridge(alpha= x/10).fit(X_train, y_train).predict(X_test)
    metrics[f'Ridge not rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

    y_test_hat = Lasso(alpha= x/10).fit(X_train, y_train).predict(X_test)
    metrics[f'Lasso not rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

    
    for r in range(1,10):
        
            y_test_hat = ElasticNet(alpha= x/10, l1_ratio = r/10).fit(X_train, y_train).predict(X_test)
            metrics[f'ElasticNet not rescaled with alpha={x/10} and r={r/10}'] = eval(y_test_hat, y_test)
            
            y_test_hat = ElasticNet(alpha= x/10, l1_ratio = r/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
            metrics[f'ElasticNet rescaled with alpha={x/10} and r={r/10}'] = eval(y_test_hat, y_test)

In [ ]:
#Models 11, 12 and 13: Boosted models


In [104]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df = pd.DataFrame.from_dict(metrics, orient='index', dtype=float, columns= ['MAE', 'MSE','relative MSE', 'R2'])
sorted_df = df.sort_values(by = ['R2'], ascending = False)
sorted_df

,MAE,MSE,relative MSE,R2
ElasticNet rescaled with alpha=0.1 and r=0.8,18591.56,834586610.99,0.02,0.87
ElasticNeT rescaled with alpha=0.2 and r=0.9,18591.12,834550417.73,0.02,0.87
ElasticNet rescaled with alpha=0.2 and r=0.9,18591.12,834550417.73,0.02,0.87
ElasticNeT rescaled with alpha=0.1 and r=0.8,18591.56,834586610.99,0.02,0.87
Lasso not rescaled with alpha=0.9,18952.31,867999790.84,0.02,0.86
...,...,...,...,...
Decision_tree_max_depth=2,36972.16,3146433453.59,0.08,0.07
KNN_9,32467.51,2953077651.94,0.07,0.03
SLR,38732.03,3107016842.50,0.08,0.03
Decision_tree_max_depth=1,45256.99,4012498211.59,0.10,-0.32
